In [105]:
import os, sys, re
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf

from tensorflow import keras


print(sys.version_info)
for module in tf, mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
tensorflow 2.1.0
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.keras 2.2.4-tf


In [154]:
DATA_DIR = './data/BX-CSV-Dump/'
class DataLoad:
    '''
    books_with_blurbs.csv cloumns: ISBN,Title,Author,Year,Publisher,Blurb
    BX-Book-Ratings.csv cloumns: User-ID,ISBN,Book-Rating
    BX-Books.csv cloumns: ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
    BX-Users.csv cloumns: User-ID,Location,Age
    '''
    def __init__(self):
        self.BX_Books = self.load_origin('BX-Books')
        self.BX_Users = self.load_origin('BX-Users')
        self.BX_Book_Ratings = self.load_origin('BX-Book-Ratings')
        self.books_with_blurbs = self.load_origin('books_with_blurbs', ',')

    def load_origin(self, 
        filename: "根据文件名获取源文件，获取正确得columns、values等值", 
        sep: "因为源文件的分隔方式sep不同，所以通过传参改编分隔方式"="\";\"", 
        )->"返回dataframe格式数据":
        try:
            data = pd.read_csv(DATA_DIR+filename+'.csv', engine='python',sep=sep, encoding='utf-8')
            def Dquote_filter(f_data):
                '''
                源文件中的columns和各个值得第一列的第一个字符和最后一列的最后一个字符都带有双引号‘"’,需要将其filter
                数据中值为NULL，我们将其替换成'<PAD>'
                '''
                f_data.rename(columns={f_data.columns[0]:f_data.columns[0][1:], f_data.columns[-1]:f_data.columns[-1][:-1]})
                f_data[f_data.columns[0]] = f_data[f_data.columns[0]].map(lambda v:v[1:] if v!=None else '<PAD>')
                f_data[f_data.columns[-1]] = f_data[f_data.columns[-1]].map(lambda v:v[:-1] if v!=None else '<PAD>')
                return f_data
            data_dict = {"\";\"":Dquote_filter(data), ',':data}
            print(data_dict[sep].head)
            return data_dict[sep]
        except UnicodeDecodeError as e:
            print('UnicodeDecodeError:',e)
origin_DATA = DataLoad()

<bound method NDFrame.head of              "ISBN                                         Book-Title  \
0       0195153448                                Classical Mythology   
1       0002005018                                       Clara Callan   
2       0060973129                               Decision in Normandy   
3       0374157065  Flu: The Story of the Great Influenza Pandemic...   
4       0393045218                             The Mummies of Urumchi   
...            ...                                                ...   
271374  0440400988                         There's a Bat in Bunk Five   
271375  0525447644                            From One to One Hundred   
271376  006008667X  Lily Dale : The True Story of the Town that Ta...   
271377  0192126040                        Republic (World's Classics)   
271378  0767409752  A Guided Tour of Rene Descartes' Meditations o...   

                 Book-Author  Year-Of-Publication  \
0         Mark P. O. Morford            